In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
from rasterio.enums import Resampling
import dask.array as da
import pylandstats as pls
import cv2
from skimage.util import view_as_blocks
import gc

Module /home/zyzh0/miniconda3/envs/gis/lib/python3.10/site-packages/pylandstats/landscape.py has not been compiled for Transonic-Pythran


In [2]:
shape_frame = gpd.read_file(
    "/home/zyzh0/coding/risk_evaluation/scripts/datas/BOARDER/extracted/华北平原地市级/华北平原地市级.shp")
boundary = shape_frame.dissolve()


In [3]:
clcd_frame = xr.open_dataset("/home/zyzh0/coding/risk_evaluation/scripts/outputs/CLCD/CLCD_v01_2015.nc")
clcd_frame.rio.write_crs("epsg:4326", inplace=True)

<xarray.Dataset>
Dimensions:                        (y: 55661, x: 48240)
Coordinates:
  * y                              (y) float64 43.0 43.0 43.0 ... 28.0 28.0 28.0
  * x                              (x) float64 110.0 110.0 110.0 ... 123.0 123.0
    spatial_ref                    int64 0
Data variables:
    __xarray_dataarray_variable__  (y, x) float64 ...

In [4]:
def mask_dataset(dataset: xr.Dataset, mask_type: int) -> xr.Dataset:
    dataset_masked = dataset.where(dataset == mask_type)
    return dataset_masked

def reproject_original_dataset(dataset: xr.Dataset, shape: tuple, resampling: Resampling = Resampling.nearest) -> xr.Dataset:
    dataset.rio.write_crs("epsg:4326", inplace=True)
    dataset_reprojected = dataset.rio.reproject(
        dataset.rio.crs, shape=shape, resampling=resampling)
    return dataset_reprojected

def get_array_from_dataset(dataset: xr.Dataset) -> np.ndarray:
    array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=True)
    return array

def patch_array(array: np.ndarray, patch_size: tuple) -> np.ndarray:
    array_patched = view_as_blocks(array, block_shape=patch_size)
    return array_patched

def get_N_number(types_array:np.ndarray) -> int:
    connectivity = 4
    out = cv2.connectedComponentsWithStats(types_array, connectivity, cv2.CV_32S)
    del types_array
    return out[0]

In [5]:
def calc_patch_number_matrix_and_save(mask_type:int,tomask_dataset:xr.Dataset):
    clcd_masked = mask_dataset(tomask_dataset, mask_type)
    print("masked")
    clcd_reprojected = reproject_original_dataset(clcd_masked, shape=(55660,48240))
    print("reprojected")
    clcd_array = get_array_from_dataset(clcd_reprojected)
    patched_clcd_array = patch_array(clcd_array, patch_size=(20,30))
    del clcd_masked , clcd_reprojected , clcd_array
    gc.collect()
    print("patched")
    test_N_number_array = np.zeros_like(patched_clcd_array[:,:,1,1])

    def calc(i,j) : test_N_number_array[i,j] = get_N_number(patched_clcd_array[i,j])

    [[calc(i,j) for i in range(patched_clcd_array.shape[0])] for j in range(patched_clcd_array.shape[1])]

    np.save(str(mask_type),test_N_number_array)

    del test_N_number_array
    gc.collect()

    return mask_type
    

In [6]:
for i in range(1,10):
    print(calc_patch_number_matrix_and_save(i,clcd_frame))

masked
reprojected


/tmp/ipykernel_25048/3724825301.py:12: RuntimeWarning: invalid value encountered in cast
  array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=True)


patched
1
masked
reprojected


/tmp/ipykernel_25048/3724825301.py:12: RuntimeWarning: invalid value encountered in cast
  array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=True)


patched
2
masked
reprojected


...

patched
7
masked
reprojected


/tmp/ipykernel_25048/3724825301.py:12: RuntimeWarning: invalid value encountered in cast
  array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=True)


patched
8
masked
reprojected


/tmp/ipykernel_25048/3724825301.py:12: RuntimeWarning: invalid value encountered in cast
  array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=True)


patched
9
